#### LIBRERIAS

In [1]:
import time
import os
import secrets
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.core.problem import Problem
from pymoo.optimize import minimize
from pymoo.operators.sampling.rnd import BinaryRandomSampling
from pymoo.operators.crossover.hux import HUX
from pymoo.operators.mutation.bitflip import BitflipMutation
from pymoo.indicators.hv import HV
from pymoo.core.sampling import Sampling
from scipy.spatial.distance import cdist
from pandas.plotting import parallel_coordinates
from matplotlib.gridspec import GridSpec
from pymoo.indicators.gd import GD
from pymoo.indicators.igd import IGD
from pymoo.termination import get_termination

#### TIME

In [2]:
class Timer:
    def __enter__(self):
        self.start = time.time()
        return self
    def __exit__(self, *args):
        self.end = time.time()
        self.interval = self.end - self.start
        self.minutes = self.interval / 60

#### ENTROPIA ESPACIAL

In [3]:
def calcular_entropia_espacial(sol, x, y, grid_size=(50,50)):
    xmin, xmax = np.min(x), np.max(x)
    ymin, ymax = np.min(y), np.max(y)

    x_bins = np.linspace(xmin, xmax, grid_size[0] + 1)
    y_bins = np.linspace(ymin, ymax, grid_size[1] + 1)

    selected_x = x[sol == 1]
    selected_y = y[sol == 1]

    if len(selected_x) == 0:
        return 0.0

    counts, _, _ = np.histogram2d(selected_x, selected_y, bins=[x_bins, y_bins])

    probs = counts.flatten() / np.sum(counts)
    probs = probs[probs > 0]

    H = -np.sum(probs * np.log2(probs))
    Hmax = np.log2(grid_size[0] * grid_size[1])
    H_norm = H / Hmax

    return H_norm

#### ENTRADA Y TRATAMIENTO PREVIO DE LOS DATOS

In [4]:
# Cargamos los datos en un DataFrame
df = pd.read_csv('Pozos_GA_datos_entrada_revisado.csv', delimiter=';', encoding='Latin-1')

# Realizamos las conversiones de los datos de entrada, debido a que los puntos decimales estan escritos con ',', los mismos los reemplazamos con '.' 
# antes de aplicar la funcion de conversion de tipo
df['WQI'] = df['WQI'].str.replace(',', '.', regex=False).astype(float)
df['ivct'] = df['ivct'].str.replace(',', '.', regex=False).astype(float)
df['ivnt'] = df['ivnt'].str.replace(',', '.', regex=False).astype(float)
df['x'] = df['x'].astype(int)
df['y'] = df['y'].astype(int)
df['prof__m_'] = df['prof__m_'].astype(int)
df['Prioridad'] = df['Prioridad'].astype(int)

#### CARGA DE LOS FRENTES IDEALES

In [5]:
def cargar_frentes_ideales(seed):

    ideales = []
    ideal_path = []

    for k in max_pozos_array:
        ideal_path.append(f"IDEALES\\seed{seed}\\k{k}-FRENTE-PARETO.csv")
    
    for x in ideal_path:
        dfx = pd.read_csv(x, delimiter=';', encoding='Latin-1')
        dfx = dfx.drop(dfx.columns[-1], axis=1)
        ideales.append(np.array(dfx))

    return ideales

#### CALLBACK

In [6]:
pareto_generations = {}

def nsga2_callback(algorithm):
    gen = algorithm.n_gen
    if gen in [100, 500, 1000, 1500, 2000]:
        # Solo el frente de Pareto no dominado
        X_pareto = algorithm.opt.get("X")
        F_pareto = algorithm.opt.get("F")
        pareto_generations[gen] = (X_pareto.copy(), F_pareto.copy())

#### DEFINICION DE LA FUNCION INTERPOLACION

In [7]:
def idw_interpolation(selected_wells):
    """ 
    Realiza una interpolación de Inverse Distance Weighting (IDW) sobre una grilla basada en una selección de pozos activos.
    - Parámetros:
        selected_wells (array): Un array binario donde 1 indica que el pozo está seleccionado y 0 indica que no está seleccionado.
    - Retorna:
        np.ndarray: Una matriz de tamaño (m_x, m_y) con los valores interpolados de WQI en la grilla definida.
    """
    active_indices = np.where(selected_wells == 1)[0]
    if len(active_indices) == 0:
        return np.zeros((m_x, m_y))
        
    active_x = x[active_indices]
    active_y = y[active_indices]
    active_WQI = WQI[active_indices]
        
    grid_points = np.c_[grid_x.ravel(), grid_y.ravel()]
    well_points = np.c_[active_x, active_y]
        
    distances = cdist(grid_points, well_points)
    weights = 1 / (distances ** 2 + 1e-10)  # Evitar división por cero
    interpolated_values = np.sum(weights * active_WQI, axis=1) / np.sum(weights, axis=1)
    return interpolated_values.reshape(m_x, m_y)

#### POBLACION INICIAL

In [8]:
class FixedOnesSampling(Sampling):
    def __init__(self, n_ones, seed=None):
        super().__init__()
        self.n_ones = n_ones
        self.rng = np.random.default_rng(seed)

    def _do(self, problem, n_samples, **kwargs):
        n_var = problem.n_var
        X = np.zeros((n_samples, n_var), dtype=bool)
        for i in range(n_samples):
            idx = self.rng.choice(n_var, self.n_ones, replace=False)
            X[i, idx] = True
        return X

#### DEFINICION DEL PROBLEMA PARA EL ALGORITMO GENETICO

In [9]:
class WellSelectionProblem(Problem):
    def __init__(self, max_pozos, prioridad, coordenadas_x, coordenadas_y, grid_size=(50, 50)):
        n_b = len(coordenadas_x)   # número total de pozos candidatos
        super().__init__(n_var=n_b, n_obj=3, n_constr=1, xl=0, xu=1, type_var=bool)
        self.max_pozos = max_pozos
        self.prioridad = prioridad
        self.x = coordenadas_x
        self.y = coordenadas_y
        self.grid_size = grid_size

    def _evaluate(self, X, out, *args, **kwargs):
        f1_vals, f2_vals, f3_vals, g_vals = [], [], [], []

        for sol in X:
            interpolated_WQI = idw_interpolation(sol)
            
            # F1: FUNCION NASH (MENOS INFINITO, 1]
            numerator = np.sum((interpolated_WQI - WQI_star_grid) ** 2)
            denominator = np.sum((WQI_star_grid - WQI_star_mean) ** 2)
            nse = 1 - numerator / denominator if denominator != 0 else -np.inf
            f1 = -nse

            # F2: PROPORCION DE POZOS PRIORITARIOS [0, 1]
            seleccionados = np.sum(sol)
            f2 = np.sum(prioridad[sol == True]) / max(seleccionados, 1)
            f2 = -f2

            # F3: ENTROPIA ESPACIAL [0, 1]
            H = calcular_entropia_espacial(sol, self.x, self.y, grid_size=self.grid_size)
            f3 = -H
            
            f1_vals.append(f1)
            f2_vals.append(f2)
            f3_vals.append(f3)
            
            # Restricción de igualdad: exactamente max_pozos
            g_vals.append(abs(seleccionados - self.max_pozos))
            
        out["F"] = np.column_stack([f1_vals, f2_vals, f3_vals])
        out["G"] = np.array(g_vals)

#### PARAMETROS GENERALES DE LA EJECUCION Y VARIABLES GOBLALES

In [10]:
# Parametros
generaciones = 2000  # Total de generaciones por cada nueva iteracion
grid_nash = 50  # Tamano de la malla a utilizar para evaluar la funcion NASH
grid_entropia = 50  # Tamano de la malla a utilizar para evaluar la funcion de la entropia espacial
n_p = 105  # Tamaño de población. Numero total de soluciones en el frente de pareto.
p_cross = 0.95  # Probabilidad de cruce
p_mut = 0.05  # Probabilidad de mutación
max_pozos_array = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]  # Maxima cantidad de pozos por individio
seed_array = [1827841104, 3893035886, 2761146642, 1589321152, 3139764589]
#seed=1827841104
#seed=3893035886
#seed=2761146642
#seed=1589321152
#seed=3139764589

#### CALCULOS PREVIOS

In [11]:
"""
x : Coordenadas en el eje X (abscisas) correspondientes a todos los pozos.
y : Coordenadas en el eje Y (ordenadas) correspondientes a todos los pozos.
WQI : Indice de calidad del agua en cada pozo.
n_b : Numero total de pozos en el conjunto de datos.
"""
x = df['x'].values
y = df['y'].values
WQI = df['WQI'].values
n_b = len(WQI)
prioridad = df['Prioridad'].values
    
# Grilla de interpolación
m_x, m_y = grid_nash, grid_nash
grid_x, grid_y = np.meshgrid(np.linspace(min(x), max(x), m_x),
                            np.linspace(min(y), max(y), m_y))

# Matriz de referencia de interpolación IDW utilizando todos los pozos.
WQI_star_grid = idw_interpolation(np.ones(n_b))
# Promedio de los valores interpolados en la grilla de referencia.
WQI_star_mean = np.mean(WQI_star_grid)

#### OUTPUT

In [12]:
def output_result(res, ideal, metricas, max_pozos, pareto_generations, directorio_k):

    for gen, (X, F) in pareto_generations.items():

        # Ruta gen
        carpeta_gen = os.path.join(directorio_k, f"GN{gen}")
        os.makedirs(carpeta_gen, exist_ok=True)

        # Ruta solus
        carpeta_solus = os.path.join(carpeta_gen, f"SOLUS")
        os.makedirs(carpeta_solus, exist_ok=True)
    
        # Resultados de la generacion actual
        pareto_front = F
        selected_solutions = X
    
        # Calculo del hipervolumen
        ref_point = [0, 0, 0]
        hv = HV(ref_point=ref_point)
        hv_value = hv(res.F)
        #print(f"\nHipervolumen (HV) para {gen} generaciones: {hv_value:.6f}")
        
        # DataFrame con los resultados
        df_results = pd.DataFrame({
            'f1': -pareto_front[:, 0],
            'f2': -pareto_front[:, 1],
            'f3': -pareto_front[:, 2],
            'cant_pozos': np.sum(selected_solutions, axis=1)
        })
    
        # Exportamos los pozos seleccionados por cada solucion del frente de pareto
        j = 1
        for i, pareto in enumerate(selected_solutions):
    
            # Filtrar los pozos seleccionados con la solución binaria
            df_pozos = df[pareto.astype(bool)].copy()
            # Obtener cantidad de pozos seleccionados
            n_pozos = df_pozos.shape[0]
            
            # Realizamos las conversiones de los datos de salida
            df_pozos['WQI'] = df_pozos['WQI'].astype(str).str.replace('.', ',', regex=False)
            df_pozos['ivct'] = df_pozos['ivct'].astype(str).str.replace('.', ',', regex=False)
            df_pozos['ivnt'] = df_pozos['ivnt'].astype(str).str.replace('.', ',', regex=False)
            df_pozos['x'] = df_pozos['x'].astype(str)
            df_pozos['y'] = df_pozos['y'].astype(str)
            df_pozos['prof__m_'] = df_pozos['prof__m_'].astype(str)
            df_pozos['Prioridad'] = df_pozos['Prioridad'].astype(str)
        
            # Construir ruta del archivo con el número de solución y cantidad de pozos
            file_path = os.path.join(carpeta_solus, f"GN{gen}_SOL{j}_NP{n_pozos}.csv")
            j = j + 1
            
            # Exportar
            df_pozos.to_csv(file_path, index=False, sep=";", encoding='UTF-8')
    
        # Inicializar variables
        best_sum = -np.inf
        best_solution = None
        best_index = -1
        
        # Normalización min-max para cada columna y creación de columnas normalizadas
        df_norm = df_results.copy()
        for col in ['f1', 'f2', 'f3']:
            df_norm[col + '_norm'] = (df_norm[col] - df_norm[col].min()) / (df_norm[col].max() - df_norm[col].min())
    
        # Calcular suma de los objetivos normalizados
        df_norm['sum_norm'] = df_norm[['f1_norm', 'f2_norm', 'f3_norm']].sum(axis=1)
    
        # Seleccionar la mejor solución directamente
        best_index = df_norm['sum_norm'].idxmax()
        best_solution = selected_solutions[best_index]
        best_sum = df_norm.loc[best_index, 'sum_norm']
    
        #print("Mejor suma normalizada:", best_sum)
        #print("Índice de la mejor solución:", best_index)
    
        # Calculo del GD e IGD
        gd = GD(ideal)
        igd = IGD(ideal)
        gd_value = gd(-pareto_front)
        igd_value = igd(-pareto_front)
        #print("GD:", gd_value)
        #print("IGD:", igd_value)
    
        # Cargamos las metricas calculadas
        metricas.append([max_pozos, gen, hv_value, gd_value, igd_value])
        
        # Exportar solo la mejor solución
        if best_solution is not None:
            df_pozos = df[best_solution.astype(bool)].copy()
            n_pozos = df_pozos.shape[0]
        
            # Realizar conversiones
            df_pozos['WQI'] = df_pozos['WQI'].astype(str).str.replace('.', ',', regex=False)
            df_pozos['ivct'] = df_pozos['ivct'].astype(str).str.replace('.', ',', regex=False)
            df_pozos['ivnt'] = df_pozos['ivnt'].astype(str).str.replace('.', ',', regex=False)
            df_pozos['x'] = df_pozos['x'].astype(str)
            df_pozos['y'] = df_pozos['y'].astype(str)
            df_pozos['prof__m_'] = df_pozos['prof__m_'].astype(str)
            df_pozos['Prioridad'] = df_pozos['Prioridad'].astype(str)
        
            # Construir nombre del archivo con el índice de solución
            file_path = os.path.join(carpeta_gen, f"GN{gen}_BESTSOL{best_index+1}_NP{n_pozos}.csv")
        
            # Exportar CSV
            df_pozos.to_csv(file_path, index=False, sep=";", encoding='UTF-8')
           
        # Imprimir tabla de resultados
        #print("\nResultados de la Frontera de Pareto:\n")
        #print(df_results.to_string(index=False))
    
        # Crear figura 2x2
        fig, axs = plt.subplots(2, 2, figsize=(16, 12))
        
        # -------------------------------
        # 1) f1 vs f2
        axs[0, 0].scatter(df_results['f2'], df_results['f1'], c='b', label='Otras')
        axs[0, 0].scatter(
            df_results.loc[best_index, 'f2'],
            df_results.loc[best_index, 'f1'],
            color='red',
            s=50,
            label='Mejor'
        )
        axs[0, 0].set_xlabel("Proporción de pozos prioritarios (f2)")
        axs[0, 0].set_ylabel("Eficiencia Nash-Sutcliffe (f1)")
        axs[0, 0].set_title("f1 vs f2")
        axs[0, 0].grid()
        axs[0, 0].legend()
        
        # -------------------------------
        # 2) f2 vs f3 (ahora en [0,1])
        axs[0, 1].scatter(df_results['f3'], df_results['f2'], c='purple', label='Otras')
        axs[0, 1].scatter(
            df_results.loc[best_index, 'f3'],
            df_results.loc[best_index, 'f2'],
            color='red',
            s=50,
            label='Mejor'
        )
        axs[0, 1].set_xlabel('Entropía Espacial (f3)')
        axs[0, 1].set_ylabel('Proporción de pozos prioritarios (f2)')
        axs[0, 1].set_title('f2 vs f3')
        axs[0, 1].grid()
        axs[0, 1].legend()
        
        # -------------------------------
        # Unir las dos de abajo: [1,0] + [1,1]
        fig.clf()  # Borrar subplots previos para usar GridSpec
        gs = GridSpec(2, 2, figure=fig)
        
        # Reubicar los de arriba
        ax1 = fig.add_subplot(gs[0, 0])
        ax2 = fig.add_subplot(gs[0, 1])
        
        # Reasignar
        ax1.scatter(df_results['f2'], df_results['f1'], c='b', label='Otras')
        ax1.scatter(
            df_results.loc[best_index, 'f2'],
            df_results.loc[best_index, 'f1'],
            color='red',
            s=50,
            label='Mejor'
        )
        ax1.set_xlabel("Proporción de pozos prioritarios (f2)")
        ax1.set_ylabel("Eficiencia Nash-Sutcliffe (f1)")
        ax1.set_title("f1 vs f2")
        ax1.grid()
        ax1.legend()
        
        ax2.scatter(df_results['f3'], df_results['f2'], c='purple', label='Otras')
        ax2.scatter(
            df_results.loc[best_index, 'f3'],
            df_results.loc[best_index, 'f2'],
            color='red',
            s=50,
            label='Mejor'
        )
        ax2.set_xlabel('Entropía Espacial (f3)')
        ax2.set_ylabel('Proporción de pozos prioritarios (f2)')
        ax2.set_title('f2 vs f3')
        ax2.grid()
        ax2.legend()
        
        # Eje grande abajo: [1,0] y [1,1] unidos
        ax3 = fig.add_subplot(gs[1, :])  # Toda la fila inferior
        
        # -------------------------------
        # Coordenadas paralelas normalizadas
        df_parallel = df_results[['f1', 'f2', 'f3']].copy()
        min_max = {}
        for col in ['f1', 'f2', 'f3']:
            min_val = df_parallel[col].min()
            max_val = df_parallel[col].max()
            df_parallel[col] = (df_parallel[col] - min_val) / (max_val - min_val)
            min_max[col] = (min_val, max_val)
        
        df_parallel['Grupo'] = 'Otras'
        df_parallel.loc[best_index, 'Grupo'] = 'Mejor'
        
        parallel_coordinates(
            df_parallel[df_parallel['Grupo'] == 'Otras'],
            class_column='Grupo',
            cols=['f1', 'f2', 'f3'],
            ax=ax3,
            colormap='viridis',
            alpha=0.7
        )
        
        ax3.plot(
            ['f1', 'f2', 'f3'],
            df_parallel.loc[best_index, ['f1', 'f2', 'f3']],
            color='red',
            linewidth=2.5,
            label='Mejor Solución'
        )
        
        ax3.legend()
        ax3.set_title("Coordenadas Paralelas")
        ax3.set_ylim(0, 1)
        ax3.set_ylabel("Valores Normalizados [0-1]")
        ax3.grid()
        
        # -------------------------------
        plt.suptitle("GRÁFICAS COMBINADAS", fontsize=16)
        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        
        # Guardar imagen combinada
        file_path = os.path.join(carpeta_gen, f"frente_combinado_2x2.png")
        plt.savefig(file_path, dpi=300)
        plt.close(fig)
        #plt.show()
    
        # DataFrame a un archivo CSV
        file_path = os.path.join(carpeta_gen, f"pareto_results.csv")
        df_results.to_csv(file_path, index=False, sep=";", encoding='UTF-8')
    
        # Ruta del archivo TXT con los parámetros generales
        file_path = os.path.join(carpeta_gen, f"parametros.txt")
    
        # Contenido del resumen de ejecución
        contenido = f"""Resumen de parámetros de ejecución - Selección de Pozos
    
            Semilla aleatoria utilizada: {seed}
            
            Parámetros del algoritmo:
            - Generaciones: {gen}
            - Tamaño de población: {n_p}
            - Probabilidad de cruce (p_cross): {p_cross}
            - Probabilidad de mutación (p_mut): {p_mut}
            - Tamaño de la malla de interpolación: {grid_nash}x{grid_nash}
            - Tamaño de la malla para el calculo de la entropia: {grid_entropia}x{grid_entropia}
            - Maxima cantidad de genes: {max_pozos}
            
            Número total de pozos en el dataset: {n_b}
            
            Punto de referencia usado para hipervolumen: {ref_point}
            Hipervolumen (HV) obtenido: {hv_value:.6f}
            GD: {gd_value}
            IGD: {igd_value}
            
            """
            
        # Guardar el archivo de texto
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(contenido)

#### CORE

In [13]:
termination = get_termination("n_gen", generaciones)

# Crear carpeta de salida si no existe
os.makedirs("OUTPUT", exist_ok=True)

for seed in seed_array:

    index = 0
    metricas = []
    tiempo = []
    now = time.localtime()
    carpeta_semilla = f"seed_{seed}_({now.tm_mday:02d}-{now.tm_mon:02d}-{now.tm_year}_{now.tm_hour:02d}-{now.tm_min:02d})"
    directorio_semilla = os.path.join("OUTPUT", carpeta_semilla)
    os.makedirs(directorio_semilla, exist_ok=True)

    for max_pozos in max_pozos_array:

        ideales = cargar_frentes_ideales(seed)

        carpeta_k = f"k_{max_pozos}"
        directorio_k = os.path.join(directorio_semilla, carpeta_k)
        os.makedirs(directorio_k, exist_ok=True)
        
        with Timer() as t:
        
            problem = WellSelectionProblem(max_pozos=max_pozos, prioridad=prioridad, coordenadas_x=x, coordenadas_y=y, grid_size=(grid_entropia,grid_entropia))
                
            algorithm = NSGA2(
                pop_size=n_p,
                sampling=FixedOnesSampling(n_ones=max_pozos, seed=seed),
                crossover=HUX(prob=p_cross),
                mutation=BitflipMutation(prob=p_mut),
                eliminate_duplicates=True
            )
            
            res = minimize(problem, algorithm, termination, seed=seed, callback=nsga2_callback, save_history=True)
            
            output_result(res, ideales[index], metricas, max_pozos, pareto_generations, directorio_k)
    
        tiempo.append(round(t.minutes, 4))
        tiempo.append(round(0, 4))
        tiempo.append(round(0, 4))
        tiempo.append(round(0, 4))
        tiempo.append(round(0, 4))
    
        index = index + 1

    # Representar metricas
    metricas = [row + [val] for row, val in zip(metricas, tiempo)]
    df_metricas = pd.DataFrame(metricas, columns=['K', 'Generacion', 'HV', 'GD', 'IGD', 'TIEMPO (MINUTOS)'])

    # Exportar metricas
    ruta_metricas = os.path.join(directorio_semilla, "metricas.csv")
    df_metricas.to_csv(ruta_metricas, index=False, sep=';', decimal='.')

C:\Users\elisfer\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\plotting\_matplotlib\misc.py:422: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  ax.legend(loc="upper right")
C:\Users\elisfer\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\plotting\_matplotlib\misc.py:422: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  ax.legend(loc="upper right")
C:\Users\elisfer\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\plotting\_matplotlib\misc.py:422: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  ax.legend(loc="upper right")
C:\Users\elisfer\AppData\Local\Programs\Python\Py